## From CSV File: Stop and Frisk / NYPD

In [ ]:
# Let's grab the csv file ('wget'), unzip it, and change its name to something that doesn't include spaces.
# SSH into your remote machine and run the following commands:

''' 
    cd /class/itpmssd/datasets
    wget http://www.nyclu.org/files/stopandfrisk/Stop-and-Frisk-2012.zip
    unzip Stop-and-Frisk-2012.zip
    mv SQF\ 2012.csv sf2012.csv
'''

#### Now let's explore the dataset using the command line first

In [ ]:
# Now we can open the file
import seaborn as sns
import pandas as pd

path = '/class/itpmssd/datasets/sf2012_sm.csv'
df = pd.read_csv(path, low_memory=False)

In [ ]:
# different ways to look at the data (since it is way too large to simply print)

df.head()

In [ ]:
df.tail(4)

In [ ]:
df[900:905]

### Cleaning the Data

In [ ]:
import datetime

# translate the messy date string into something cleaner -> a datetime structure
# (M)MDDYYYY

def parse_hour(timestop):
    minute = timestop % 100
    hour = timestop / 100
    return hour,minute

def parse_date(datestop):
    month = int(str(datestop)[:-6])
    day = int(str(datestop)[-6:-4])
    year = int(str(datestop)[-4:])
    return year, month, day

def make_datetime(datestop, timestop):
    year, month, day = parse_date(datestop)
    hour, minute = parse_hour(timestop)
    return datetime.datetime(year, month, day, hour)

def make_date(datestop):
    year, month, day = parse_date(datestop)
    return datetime.datetime(year, month, day)

In [ ]:
print parse_date(df.ix[0].datestop)
print parse_hour(df.ix[0].timestop)
print make_datetime(df.ix[0].datestop, df.ix[0].timestop)
print make_date(df.ix[0].datestop)

In [ ]:
# now lets add a parsed datetime row in our data frame - by applying the function that we've just built
# this will give us a 'dt' field we can use to group by

df['dt']=df[['datestop','timestop']].apply(lambda x: make_datetime(x['datestop'], x['timestop']), axis=1)
df['d']=df[['datestop']].apply(lambda x: make_date(x['datestop']), axis=1)

In [ ]:
df.tail()

### Grouping: split-apply-combine

In [ ]:
# Grouping - counting the number of reported incidents per hour (can also do grouped.count())

df.groupby(df.dt).size()[:10]

In [ ]:
# let's make a plot - that shows trends over time
%pylab inline

df.groupby(df.dt).size().plot(figsize=(20,6))

In [ ]:
# we can easily specify a smaller date range
df.groupby(df.dt).size()[:'2012-02-16'].plot(figsize=(20,6))

### Let's examine age

In [ ]:
# apparently immortality is imminent

sns.boxplot(df.age)

In [ ]:
# DRAAAAATS - more weirdnesses in the data!

In [ ]:
# argmax prints the row # for the maximum value in a given column
print 'maximum age index number:',df.age.argmax(),'\n'

In [ ]:
# ix[] -> is a way to access values given a specific row
print 'maximum age row:',df.ix[1021]

In [ ]:
# How bad is it? How many rows in our data have weird age figures?
MAX_AGE = 100

print 'number of problematic rows:',len(df[[a>MAX_AGE for a in df.age]])

# 0.2% of our data is problematic
len(df[[a>MAX_AGE for a in df.age]])/float(len(df))


In [ ]:
# filter out rows with faulty age info
df = df[[x<MAX_AGE for x in df.age]]

In [ ]:
# Let's try this again
sns.boxplot(df.age)

### Filter out Columns

In [ ]:
# we can always get rid of unnecessary columns - makes DataFrames easier to handle

wanted_columns = ['dt','d','age','sex','race','height','weight','build','frisked']

sm_df = df[wanted_columns]
sm_df.head()

### Now let's take a look at Race

In [ ]:
race_labels = {
    1:'black', 
    2:'black_Hispanic', 
    3:'white_Hispanic', 
    4:'white', 
    5:'Asian_Pacific_Islander', 
    6:'Am_Indian_Native_Alaskan'
}

In [ ]:
sm_df.race[10:20]

In [ ]:
# we can use our handy dictionary to apply the labels
[race_labels[x] for x in sm_df.race[10:20]]

In [ ]:
# Handy way to count values in a column

sm_df.race.value_counts()

In [ ]:
# histogram: show entries by race
sm_df.race.hist(figsize=(12,6))
ylabel('number of incidents')
xlabel('race')
title('incidents by race')

In [ ]:
grouped = sm_df.groupby('race')
black = grouped.get_group(1)
white_hispanic = grouped.get_group(3)
white = grouped.get_group(4)

In [ ]:
black.describe()

In [ ]:
%pylab inline
grouped.get_group(1).hist(bins=30,figsize=(14,8))

In [ ]:
# plot frisks broken down by race over time
black.groupby('dt').frisked.size()[:'2012-02-16'].plot(figsize=(20,6), label='black')
white_hispanic.groupby('dt').frisked.size()[:'2012-02-16'].plot(figsize=(20,6), label='white hispanic')
white.groupby('dt').frisked.size()[:'2012-02-16'].plot(figsize=(20,6), label='white')
legend()
title('frisked over time, by race')
ylabel('number of frisks')

In [ ]:
# multi-bar charts (note: they are overlayed on top of each other)

black.groupby('d').size()[:'2012-02-16'].plot(kind="bar", figsize=(20,6), label='black')
white_hispanic.groupby('d').size()[:'2012-02-16'].plot(kind="bar", color='red', figsize=(20,6), label='white_hispanic')
white.groupby('d').size()[:'2012-02-16'].plot(kind="bar", color='orange', figsize=(20,6), label='white')
xticks(rotation=45)
legend()
title('daily incidents, by race')

In [ ]:
# stacked bar chart

sm_df.groupby(['d','race']).size()[:'2012-02-16'].unstack('race').plot(kind="bar", figsize=(20,6), stacked=True)

xticks(rotation=45)
legend()
title('daily incidents, by race')

In [ ]:
# frisked vs. not frisked - from the population of incidents involving black people

black.groupby(['d','frisked']).size()[:'2012-02-16'].unstack('frisked').plot(kind="bar", figsize=(18,6))
title('daily frisks, for race=black')

### Correlation

In [ ]:
sm_df.corr()

In [ ]:
plt.subplots(figsize=(12, 10))
sns.corrplot(sm_df, annot=False, diag_names=False)

### Pickle the small DataFrame (for when our system crashes)

In [ ]:
# save the data for later
import pickle

f = open('/class/itpmssd/Week_2/sm_df.p','wb')
pickle.dump(sm_df,f)
f.close()

In [ ]:
# load the data from pickled file

pkl_file = open('/class/itpmssd/Week_2/sm_df.p','rb')
sm_df = pickle.load(pkl_file)

# ... aaand VOILA!

### Create Smaller Stop-and-Frisk File/DataFrame

In [ ]:
# Create Smaller File -> so that we can look at longer term trends
import csv
import pandas as pd

columns = ['datestop','timestop','sex','race','age','height','weight','build','frisked']
wanted_indices = set([3,4,6,7,9,10,11,14,16])

columns = {
    3:'datestop',
    4:'timestop',
    6:'sex',
    7:'race',
    9:'age',
    10:'height',
    11:'weight',
    14:'build',
    16:'frisked'
}

small_dict = {c:[] for c in columns.values()}

cnt = 0
with open("/class/itpmssd/datasets/sf2012.csv",'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if cnt==0:
            # dismiss the first row of the CSV file -> column headers
            cnt += 1
            continue
        
        # parse the date from the given string
        cur_date = calc_datetime(row[3])
        
        for k,c in columns.items():
            if c=='datestop':
                small_dict[c].append(cur_date)
            else:
                if row[k]!='':
                    # add relevant value to the dictionary
                    small_dict[c].append(int(row[k]))
                else:
                    # add zeros where there's no data -> generally messes up our distribution estimation
                    small_dict[c].append(0)
        
        cnt += 1

In [ ]:
sm_df = pd.DataFrame.from_dict(small_dict)
sm_df.head()

### Related Reads and References

- Precinct or Prejudice? understanding racial disparities in New York City's stop-and-frisk policy: https://5harad.com/papers/frisky.pdf
